<a href="https://colab.research.google.com/github/KNU-BrainAI-Capstone2022/ITS/blob/main/review_final2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install wordcloud
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 13.8 MB/s 
     |████████████████████████████████| 7.6 MB 60.6 MB/s 
     |████████████████████████████████| 163 kB 63.5 MB/s 


In [2]:
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch.nn as nn
import torch.optim as optim

from transformers import BertTokenizer, BertForSequenceClassification

from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import seaborn as sns

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print(device)

cuda


In [5]:
path = '/content/drive/MyDrive/processed_data.csv'

df = pd.read_csv(path)
print(df.shape)
df.head()

(8245, 5)


,Unnamed: 0,category,rating,label,text_
0,3,Home_and_Kitchen_5,1,CG,"Missing information on how to use it, but it i..."
1,13,Home_and_Kitchen_5,1,CG,These are so flimsy! They are not the quality ...
2,26,Home_and_Kitchen_5,2,CG,Not what I am accustomed to. The only reason I...
3,39,Home_and_Kitchen_5,1,CG,"Awful experience, everything stuck, cooked eve..."
4,41,Home_and_Kitchen_5,2,CG,SHEET COLOR IS NICE BUY FOR MY SIZE FOR MY TOW...


In [6]:
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

Downloading:   0%|          | 0.00/213k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [7]:
labels = {'CG':0, # 가짜면 0
          'OR':1 # 진짜면 1
          }

class Dataset(torch.utils.data.Dataset):

    def __init__(self, df):

        self.labels = [labels[label] for label in df['label']] # df 컬럼 잘 보고 설정
        self.texts = [tokenizer(text, 
                               padding='max_length', max_length = 128, truncation=True,
                                return_tensors="pt") for text in df['text_']] # df 컬럼 잘 보고 설정

    def classes(self):
        return self.labels

    def __len__(self):
        return len(self.labels)

    def get_batch_labels(self, idx):
        # Fetch a batch of labels
        return np.array(self.labels[idx])

    def get_batch_texts(self, idx):
        # Fetch a batch of inputs
        return self.texts[idx]

    def __getitem__(self, idx):

        batch_texts = self.get_batch_texts(idx)
        batch_y = self.get_batch_labels(idx)

        return batch_texts, batch_y

In [8]:
print(len(df))

8245


In [9]:
from torch import nn
from transformers import BertModel

class BertClassifier(nn.Module):

    def __init__(self, dropout=0.5):

        super(BertClassifier, self).__init__()

        self.bert = BertModel.from_pretrained('bert-base-cased') # 'bert-base-cased': 영어용, 다른 언어의 데이터면 다른거 쓰기
        self.dropout = nn.Dropout(dropout)
        self.linear = nn.Linear(768, 2)
        self.relu = nn.ReLU()

    def forward(self, input_id, mask):

        _, pooled_output = self.bert(input_ids= input_id, attention_mask=mask,return_dict=False)
        # _: embedding vectors of all of the tokens in a sequence
        # pooled_output: embedding vector of [CLS] token, 분류 문제에선 이것만 써도 충분
        dropout_output = self.dropout(pooled_output)
        linear_output = self.linear(dropout_output)
        final_layer = self.relu(linear_output)

        return final_layer

In [10]:
model = BertClassifier()

Downloading:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [11]:
result=[]
def evaluate(model, data):

    test = Dataset(data)

    dataloader = torch.utils.data.DataLoader(test, batch_size=1)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")
    if use_cuda:

        model = model.cuda()
    path = "/content/drive/MyDrive/ClassifierModel.pth"
    model.load_state_dict(torch.load(path))
    total_acc_test = 0
    with torch.no_grad():

        for input, label in dataloader:

              label = label.to(device)
              mask = input['attention_mask'].to(device)
              input_id = input['input_ids'].squeeze(1).to(device)

              output = model(input_id, mask)
              if(output.argmax(dim=1)[0]==1):
                result.append("OR")
              else:
                result.append("CG")
              
              acc = (output.argmax(dim=1) == label).sum().item()
              total_acc_test += acc
    
    print(result)
    print(len(data))
    print(total_acc_test)
    print(f'Test Accuracy: {total_acc_test / len(data): .3f}')
    
 
evaluate(model, df)


['CG', 'CG', 'CG', 'CG', 'CG', 'CG', 'CG', 'OR', 'OR', 'OR', 'OR', 'OR', 'OR', 'CG', 'CG', 'CG', 'CG', 'CG', 'OR', 'OR', 'OR', 'CG', 'CG', 'CG', 'CG', 'CG', 'OR', 'OR', 'OR', 'OR', 'OR', 'CG', 'CG', 'CG', 'CG', 'OR', 'OR', 'OR', 'OR', 'OR', 'CG', 'CG', 'CG', 'CG', 'OR', 'CG', 'OR', 'OR', 'OR', 'OR', 'OR', 'CG', 'CG', 'CG', 'OR', 'OR', 'OR', 'OR', 'OR', 'CG', 'CG', 'CG', 'CG', 'OR', 'OR', 'OR', 'CG', 'OR', 'OR', 'OR', 'OR', 'CG', 'CG', 'CG', 'OR', 'OR', 'OR', 'OR', 'CG', 'CG', 'CG', 'OR', 'OR', 'CG', 'CG', 'CG', 'OR', 'OR', 'OR', 'CG', 'CG', 'CG', 'OR', 'OR', 'CG', 'CG', 'OR', 'OR', 'OR', 'OR', 'OR', 'CG', 'CG', 'OR', 'OR', 'OR', 'OR', 'OR', 'CG', 'OR', 'OR', 'CG', 'CG', 'CG', 'CG', 'CG', 'CG', 'CG', 'OR', 'OR', 'OR', 'OR', 'CG', 'CG', 'CG', 'CG', 'OR', 'CG', 'OR', 'OR', 'CG', 'CG', 'OR', 'CG', 'CG', 'CG', 'CG', 'OR', 'OR', 'CG', 'CG', 'CG', 'CG', 'CG', 'OR', 'OR', 'OR', 'CG', 'CG', 'CG', 'CG', 'OR', 'OR', 'CG', 'CG', 'CG', 'OR', 'OR', 'OR', 'OR', 'OR', 'OR', 'OR', 'CG', 'CG', 'CG', 'CG

In [12]:
df['predicted_label'] = result
print(df.shape)
df.head()

(8245, 6)


,Unnamed: 0,category,rating,label,text_,predicted_label
0,3,Home_and_Kitchen_5,1,CG,"Missing information on how to use it, but it i...",CG
1,13,Home_and_Kitchen_5,1,CG,These are so flimsy! They are not the quality ...,CG
2,26,Home_and_Kitchen_5,2,CG,Not what I am accustomed to. The only reason I...,CG
3,39,Home_and_Kitchen_5,1,CG,"Awful experience, everything stuck, cooked eve...",CG
4,41,Home_and_Kitchen_5,2,CG,SHEET COLOR IS NICE BUY FOR MY SIZE FOR MY TOW...,CG


In [13]:
df2 = df.loc[(df['predicted_label'] == 'OR') & (df['rating'] != 3) ]
print(df2.shape)

(4208, 6)


In [20]:
df2['division'] = np.select([df2.rating > 3], [1], default=0)
positive = len(df2.loc[df2['rating'] > 3])
negative = len(df2.loc[df2['rating'] < 3])
print(f'Positive review : ', positive)
print(f'Negative review : ', negative)
df2[0:5]


Positive review :  2100
Negative review :  2108


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,Unnamed: 0,category,rating,label,text_,predicted_label,division
7,64,Home_and_Kitchen_5,1,OR,"Super rough, not soft wash cloths, more like b...",OR,0
8,67,Home_and_Kitchen_5,1,OR,Does not do a very good job; difficult to use.,OR,0
9,87,Home_and_Kitchen_5,1,OR,"sheds all over, would not buy it ever again. b...",OR,0
10,95,Home_and_Kitchen_5,2,OR,These are so weak. They barely bring in any air.,OR,0
11,105,Home_and_Kitchen_5,1,OR,Stopped working after one use still waiting on...,OR,0


In [15]:
class Dataset(torch.utils.data.Dataset):

    def __init__(self, df2):

        self.divisions = [division for division in df2['division']]
        self.texts = [tokenizer(text, 
                               padding='max_length', max_length = 128, truncation=True,
                                return_tensors="pt") for text in df2['text_']]
    def classes(self):
        return self.divisions

    def __len__(self):
        return len(self.divisions)

    def get_batch_divisions(self, idx):
        # Fetch a batch of labels
        return np.array(self.divisions[idx])

    def get_batch_texts(self, idx):
        # Fetch a batch of inputs
        return self.texts[idx]

    def __getitem__(self, idx):

        batch_texts = self.get_batch_texts(idx)
        batch_y = self.get_batch_divisions(idx)

        return batch_texts, batch_y

df2[:5]

,Unnamed: 0,category,rating,label,text_,predicted_label,division
7,64,Home_and_Kitchen_5,1,OR,"Super rough, not soft wash cloths, more like b...",OR,0
8,67,Home_and_Kitchen_5,1,OR,Does not do a very good job; difficult to use.,OR,0
9,87,Home_and_Kitchen_5,1,OR,"sheds all over, would not buy it ever again. b...",OR,0
10,95,Home_and_Kitchen_5,2,OR,These are so weak. They barely bring in any air.,OR,0
11,105,Home_and_Kitchen_5,1,OR,Stopped working after one use still waiting on...,OR,0


In [21]:
def evaluate(model, test_data):

    test = Dataset(test_data)

    test_dataloader = torch.utils.data.DataLoader(test, batch_size=1)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    if use_cuda:

        model = model.cuda()
    checking_num1 = 0
    checking_num2 = 0
    checking_num3 = 0
    checking_num4 = 0
    path = "/content/drive/MyDrive/NetModel_s.pth"
    model.load_state_dict(torch.load(path))    
    total_acc_test = 0
    with torch.no_grad():

        for test_input, test_division in test_dataloader:

              test_division = test_division.to(device)
              mask = test_input['attention_mask'].to(device)
              input_id = test_input['input_ids'].squeeze(1).to(device)

              output = model(input_id, mask)
              if output.argmax(dim=1)[0] == 1:
                  checking_num1 +=1
              elif output.argmax(dim=1)[0] == 0:
                  checking_num3 +=1              
              else:
                  checking_num4 +=1
              checking_num2+=1
              acc = (output.argmax(dim=1) == test_division).sum().item()
              total_acc_test += acc
    
    print(f'Test Accuracy: {total_acc_test / len(test_data): .3f}')
    
    print(f'Test Accuracy: {total_acc_test / len(test_data): .3f}')
    print('==',total_acc_test)
    print('==',len(test_data))
    print(checking_num1)
    print(checking_num2)
    print(checking_num3)
    print(checking_num4)
    print(f'Positive review : ', checking_num1)
    print(f'Total review : ', checking_num2)
    print(f'Positive review rate: {(checking_num1/checking_num2)*100: .3f}')
evaluate(model, df2)

Test Accuracy:  0.970
Test Accuracy:  0.970
== 4081
== 4208
2093
4208
2115
0
Positive review :  2093
Total review :  4208
Positive review rate:  49.739
